In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://myanimelist.net/anime/season/2024/spring"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
anime_list = []
anime_elements = soup.find_all('div', class_='seasonal-anime')

In [4]:
for anime in anime_elements:
    title = anime.find('h2', class_='h2_anime_title').get_text(strip=True) if anime.find('h2', class_='h2_anime_title') else 'N/A'
    release_date = anime.find('span', class_='item').get_text(strip=True) if anime.find('span', class_='item') else 'N/A'
    
    episode_info = anime.find_all('span', class_='item')
    if len(episode_info) > 1:
        total_episodes_text = episode_info[1].get_text(strip=True)
        total_episodes = total_episodes_text.split()[0] if total_episodes_text else 'N/A'
    else:
        total_episodes = 'N/A'
        
    genres = ', '.join([genre.get_text(strip=True) for genre in anime.find_all('span', class_='genre')]) if anime.find_all('span', class_='genre') else 'N/A'
    
    properties_div = anime.find('div', class_='properties')
    if properties_div:
        spans = properties_div.find_all('span', class_='item')
        studio = spans[0].get_text(strip=True) if len(spans) > 0 else 'N/A'
        source = spans[1].get_text(strip=True) if len(spans) > 1 else 'N/A'
        themes = ', '.join([span.get_text(strip=True) for span in spans[2:-1]]) if len(spans) > 2 else 'N/A'
        demographic = spans[-1].get_text(strip=True) if len(spans) > 2 else 'N/A'
    else:
        studio = 'N/A'
        source = 'N/A'
        themes = 'N/A'
        demographic = 'N/A'
    
    members = anime.find('div', class_='scormem-item member').get_text(strip=True) if anime.find('div', class_='scormem-item member') else 'N/A'
    
    score_div = anime.find('div', class_='scormem-container')
    score = score_div.get_text(strip=True) if score_div else 'N/A'
    # Extract and format score with comma after the first digit
    if score != 'N/A':
        digits = ''.join(filter(str.isdigit, score))
        score = f"{digits[0]}.{digits[1:3]}" if len(digits) >= 4 else f"{digits}"
    else:
        score = 'N/A' 
     
    anime_list.append({
        'Title': title,
        'Release Date': release_date,
        'Total Episode': total_episodes,
        'Genres': genres,
        'Studio': studio,
        'Source': source,
        'Themes': themes,
        'Demographic': demographic,
        'Score' : score,
        'Members': members
    })

In [5]:
df = pd.DataFrame(anime_list)
df.to_csv('anime_season_data.csv', index=False)